In [28]:
from keras.models import load_model
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications import imagenet_utils
from keras import applications
import numpy as np
import operator
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense

In [31]:
num_classes=10
path_top_model_weights = "bottleneck_fc_model.h5"
target_size=(224, 224)
img_path = "../dataset/split_data/test/c0/img_41813.jpg"
class_labels = ['safe_driving', 'texting_right', 'talking_on_phone_right', 'texting_left', 'talking_on_phone_left',
                'operating_radio', 'drinking', 'reaching_behind', 'doing_hair_makeup', 'talking_to_passanger']

In [21]:
# prepare image for classification using keras utility functions
image = load_img(img_path, target_size=target_size)

image_arr = img_to_array(image) # convert from PIL Image to NumPy array
# the dimensions of image should now be (224, 224, 3)

image_arr /= 255 # re-scale from [0-225] to [0-1]

# to be able to pass it through the network and use batches, we want it with shape (1, 224, 224, 3)
image_arr = np.expand_dims(image_arr, axis=0)

In [23]:
# build the VGG16 network
model = applications.VGG16(include_top=False, weights='imagenet')

# get the bottleneck prediction from the pre-trained vgg16 model
bottleneck_prediction = model.predict(image_arr)

In [35]:
# build the top model
model = Sequential()
model.add(Flatten(input_shape=bottleneck_prediction.shape[1:])) # use bottleneck prediction as input
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

# get weights from trained top model
model.load_weights(path_top_model_weights)

In [30]:
# use the bottleneck prediction on the top model to get the final classification
class_predicted = model.predict_classes(bottleneck_prediction)

In [ ]:
# classify given an image
predictions = model.predict(image_arr)

# get human-readable labels of the preditions, as well as the corresponding probability
decoded_predictions = dict(zip(class_labels, predictions[0]))

# sort dictionary by value
decoded_predictions = sorted(decoded_predictions.items(), key=operator.itemgetter(1), reverse=True)

# print image
plt.imshow(image)
plt.axis('off')
plt.show()

count = 1
for key, value in decoded_predictions[:5]:
    print("{}. {}: {:8f}%".format(count, key, value*100))
    count+=1